In [1]:
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import sqlite3

In [2]:
def divide(string):
    if 'https' in string:
        link = re.findall('<a href="([0-9a-z\./:-]*)">', string)[0]
        name = re.findall('>([A-ZÁÉÍÓÚÑa-záéíóúñüÜ ]*)<', string)[0].strip()
        return [name, link]
    else:
        name = re.findall('([A-ZÁÉÍÓÚÑa-záéíóúñ ]*)', string)[0].strip()
        return [name]

In [ ]:
url = 'https://masteres.ugr.es/fisymat/docencia/plan-estudios'
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")
print(html)

# Subject names scraping

In [ ]:
subjects_pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ:\n ]*)</td>'''
subjects = [x.replace('\n', '').strip() for x in re.findall(subjects_pattern, html)]
subjects

# Professors and e-mails scraped

In [ ]:
elements = re.findall('<li>([a-zA-ZÁÉÍÓÚáéíóúñüÜ0-9 <>/:/\.\-\="\n]*)</li>', html)
elements = [x.strip().replace('\n', '') for x in elements]
elements = [re.split('</li> *<li>', x) for x in elements]
for i in range(len(elements)):
    elements.extend(elements[0])
    elements.remove(elements[0])
elements = [x.strip().replace('  ', ' ') for x in elements]
elements = elements[3:]
elements = [divide(x) for x in elements]
emails = []
for i in range(len(elements)):
    try:
        emails.append(elements[i][1])
    except:
        emails.append(np.nan)
        
names = pd.DataFrame([x[0] for x in elements], columns=['Name'])
emails = pd.DataFrame(emails, columns=['E-Mail'])

data = pd.concat([names, emails], axis=1)
data

# Complete transaction

In [ ]:
url = 'https://masteres.ugr.es/fisymat/docencia/plan-estudios'
page = urlopen(url)
# html_bytes = page.read()
# html = html_bytes.decode("utf-8")
print(page.read().decode())

In [ ]:
pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ\n \.,:]*)</td>[0-9a-zA-ZáéíóúÁÉÍÓÚÑñüÜ<>/"=_\- ]*<ul> *<li>([a-zA-ZÁÉÍÓÚáéíóúñüÜ0-9 <>/:/\.\-\="\n]*)</li> *</ul>'''
result = re.findall(pattern, html.replace('\n', ''))
elements = []
for i in range(len(result)):
    elements.append(list(result[i]))
del result
for i in range(len(elements)):
    for j in range(len(elements[i])):
        elements[i][j] = elements[i][j].strip()
for i in range(len(elements)):
    elements[i][1] = re.split('</li> *<li>', elements[i][1])
for i in range(len(elements)):
    appender = []
    for j in range(len(elements[i][1])):
        appender.append(divide(elements[i][1][j]))
    elements[i][1] = appender
elements

In [ ]:
data = pd.DataFrame(columns=['Subject', 'Professor Name', 'Professor E-Mail'])
k = 0
for i in range(len(elements)):
    subject = elements[i][0]
    for j in range(len(elements[i][1])):
        try:
            email = elements[i][1][j][1]
        except:
            email = np.nan
        data.loc[k] = [subject, elements[i][1][j][0], email]
        k += 1
data

## Save to database and read

In [ ]:
data.to_sql('subjects_data', con=sqlite3.connect('data.db'))

In [ ]:
con = sqlite3.connect('data.db')
pd.read_sql('SELECT * FROM subjects_data', con=con)